In [2]:
# Visual 3: Income and Wage Distribution Comparison (Austin vs U.S., Texas vs U.S.)
import pandas as pd
import re
from pathlib import Path
from typing import Optional, Tuple, Dict, List
import plotly.graph_objects as go
from plotly.subplots import make_subplots

ORANGE = "#f28e2b"   # Austin/Texas
BLUE   = "#4e79a7"   # U.S.

def find_csv(candidates: List[Path]) -> Optional[Path]:
    for p in candidates:
        if p.exists():
            return p
    return None

def _first_col_match(cols: List[str], patterns: List[str]) -> Optional[str]:
    for pat in patterns:
        for c in cols:
            if re.search(pat, c, flags=re.I):
                return c
    return None

def _ensure_share(df: pd.DataFrame, value_col: str, grp_cols: List[str]) -> pd.Series:
    denom = df.groupby(grp_cols)[value_col].transform("sum")
    return df[value_col] / denom

def _extract_lower_bound(bucket: str) -> float:
    if not isinstance(bucket, str):
        return float("inf")
    nums = re.findall(r"\d[\d,]*", bucket)
    if not nums:
        if "less" in bucket.lower() or "<" in bucket:
            return 0.0
        return float("inf")
    try:
        return float(nums[0].replace(",", ""))
    except:
        return float("inf")

def order_categories(values: List[str], preferred: Optional[List[str]] = None) -> List[str]:
    if preferred:
        pref_index = {b: i for i, b in enumerate(preferred)}
        return sorted(set(values), key=lambda b: pref_index.get(b, 10_000 + _extract_lower_bound(b)))
    return sorted(set(values), key=_extract_lower_bound)

def load_income_data() -> pd.DataFrame:
    candidates = [
        Path("data files/austin_household_income.csv"),
        Path("data files/austin_income1_cleaned.csv"),
        Path("austin_household_income.csv"),
        Path("austin_income1_cleaned.csv"),
    ]
    p = find_csv(candidates)
    if p is None:
        raise FileNotFoundError(
            "Income dataset not found. Place 'austin_household_income.csv' or 'austin_income1_cleaned.csv' in project root or 'data files/'."
        )
    df = pd.read_csv(p)
    cols = list(df.columns)

    bucket_col = _first_col_match(cols, [r"household\s*income\s*bucket", r"\bbucket\b"])
    year_col   = _first_col_match(cols, [r"^year$"])
    region_col = _first_col_match(cols, [r"^place$", r"^region$"])
    share_col  = _first_col_match(cols, [r"^share$"])
    value_col  = _first_col_match(cols, [r"^household\s*income$"])

    if not bucket_col or not year_col or not region_col:
        raise ValueError(f"Income CSV missing required columns (bucket/year/place). Found: {cols}")

    df = df.rename(columns={
        bucket_col: "bucket",
        year_col: "year",
        region_col: "region",
        (share_col or "share"): "share",
    })

    df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")
    df = df.dropna(subset=["year"]).copy()
    df["year"] = df["year"].astype(int)

    if "share" not in df.columns or df["share"].isna().any():
        if not value_col:
            raise ValueError("Income CSV missing 'share' and no count column to compute it.")
        df["share"] = _ensure_share(df, value_col, ["year", "region"])

    keep = df["region"].isin(["Austin, TX", "United States"])
    if not keep.any():
        raise ValueError("Income CSV must contain both 'Austin, TX' and 'United States'.")
    return df[keep][["year", "region", "bucket", "share"]].copy()

def load_wage_data() -> Optional[pd.DataFrame]:
    # Expected columns: bucket | year | region | share (or counts convertible to share)
    candidates = [
        Path("data files/tx_wage_distribution.csv"),
        Path("data files/texas_wage_distribution.csv"),
        Path("texas_wage_distribution.csv"),
        Path("wage_distribution.csv"),
    ]
    p = find_csv(candidates)
    if p is None:
        return None

    df = pd.read_csv(p)
    cols = list(df.columns)
    # Expanded detection to match variants like 'Wage Bin', 'State', 'Record Count'
    bucket_col = _first_col_match(cols, [r"wage\s*(bucket|bin|range)", r"\bbucket\b", r"\bbin\b", r"^range$"])
    year_col   = _first_col_match(cols, [r"^year$"])
    region_col = _first_col_match(cols, [r"^region$", r"^place$", r"^state$", r"^geography$", r"^geo$"])
    share_col  = _first_col_match(cols, [r"^share$"])
    count_col  = _first_col_match(cols, [r"^count$", r"^record\s*count$", r"^workers?$", r"^employment$", r"^total\s*employment$", r"^employment\s*count$"])

    if not bucket_col or not year_col or not region_col:
        raise ValueError(f"Wage CSV missing required columns (bucket/year/region). Found: {cols}")

    df = df.rename(columns={
        bucket_col: "bucket",
        year_col: "year",
        region_col: "region",
        (share_col or "share"): "share",
    })

    df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")
    df = df.dropna(subset=["year"]).copy()
    df["year"] = df["year"].astype(int)

    if "share" not in df.columns or df["share"].isna().any():
        if not count_col:
            raise ValueError("Wage CSV missing 'share' and no count column to compute it.")
        df[count_col] = pd.to_numeric(df[count_col], errors="coerce").fillna(0)
        df["share"] = _ensure_share(df, count_col, ["year", "region"])

    keep = df["region"].isin(["Texas", "United States"])
    return df[keep][["year", "region", "bucket", "share"]].copy()

INCOME_ORDER = [
    "< $10,000", "$10,000 to $14,999", "$15,000 to $19,999", "$20,000 to $24,999",
    "$25,000 to $29,999", "$30,000 to $34,999", "$35,000 to $39,999", "$40,000 to $49,999",
    "$50,000 to $59,999", "$60,000 to $69,999", "$70,000 to $79,999", "$80,000 to $89,999",
    "$90,000 to $99,999", "$100,000 to $124,999", "$125,000 to $149,999",
    "$150,000 to $199,999", "$200,000+",
]

income_df = load_income_data()
wage_df = load_wage_data()  # may be None

income_buckets = order_categories(income_df["bucket"].tolist(), preferred=INCOME_ORDER)
wage_buckets = order_categories(wage_df["bucket"].tolist()) if wage_df is not None else []

income_years = sorted(income_df["year"].unique())
if wage_df is not None:
    years = sorted(set(income_years).intersection(set(wage_df["year"].unique())))
    if not years:
        years = income_years
else:
    years = income_years

def series_for(df: pd.DataFrame, year: int, buckets: List[str], region_a: str, region_b: str) -> Tuple[List[str], List[float], List[float]]:
    d = df[df["year"] == year].copy()
    pivot = d.pivot_table(index="bucket", columns="region", values="share", aggfunc="mean").fillna(0.0)
    x = buckets
    a = [float(pivot.loc[b, region_a]) if (b in pivot.index and region_a in pivot.columns) else 0.0 for b in x]
    u = [float(pivot.loc[b, region_b]) if (b in pivot.index and region_b in pivot.columns) else 0.0 for b in x]
    return x, a, u

has_wages = wage_df is not None and len(wage_buckets) > 0
row_heights = [0.6, 0.4] if has_wages else [1.0, 0.0]

fig = make_subplots(
    rows=2, cols=1, shared_x=False, vertical_spacing=0.12,
    row_heights=row_heights,
    subplot_titles=(f"Household Income Distribution — {years[0]}", "Wage Distribution — " + (str(years[0]) if has_wages else "No wage data found"))
)

# Initial traces
x_i, y_austin, y_us_income = series_for(income_df, years[0], income_buckets, "Austin, TX", "United States")
fig.add_trace(
    go.Bar(
        x=x_i, y=y_austin, name="Austin, TX", marker_color=ORANGE,
        hovertemplate="%{x}<br>Austin, TX: %{y:.1%}<extra></extra>"
    ),
    row=1, col=1
)
fig.add_trace(
    go.Bar(
        x=x_i, y=y_us_income, name="United States", marker_color=BLUE,
        hovertemplate="%{x}<br>United States: %{y:.1%}<extra></extra>"
    ),
    row=1, col=1
)

if has_wages:
    x_w, y_tx, y_us_wage = series_for(wage_df, years[0], wage_buckets, "Texas", "United States")
    fig.add_trace(
        go.Bar(
            x=x_w, y=y_tx, name="Texas", marker_color=ORANGE,
            hovertemplate="%{x}<br>Texas: %{y:.1%}<extra></extra>"
        ),
        row=2, col=1
    )
    fig.add_trace(
        go.Bar(
            x=x_w, y=y_us_wage, name="United States", marker_color=BLUE,
            hovertemplate="%{x}<br>United States: %{y:.1%}<extra></extra>"
        ),
        row=2, col=1
    )

# Frames
frames = []
for yr in years:
    data = []
    x_i, y_austin, y_us_income = series_for(income_df, yr, income_buckets, "Austin, TX", "United States")
    data.append(go.Bar(x=x_i, y=y_austin, marker_color=ORANGE, hovertemplate="%{x}<br>Austin, TX: %{y:.1%}<extra></extra>"))
    data.append(go.Bar(x=x_i, y=y_us_income, marker_color=BLUE, hovertemplate="%{x}<br>United States: %{y:.1%}<extra></extra>"))
    if has_wages:
        x_w, y_tx, y_us_wage = series_for(wage_df, yr, wage_buckets, "Texas", "United States")
        data.append(go.Bar(x=x_w, y=y_tx, marker_color=ORANGE, hovertemplate="%{x}<br>Texas: %{y:.1%}<extra></extra>"))
        data.append(go.Bar(x=x_w, y=y_us_wage, marker_color=BLUE, hovertemplate="%{x}<br>United States: %{y:.1%}<extra></extra>"))
    frames.append(go.Frame(data=data, name=str(yr), traces=list(range(0, 4 if has_wages else 2))))

fig.frames = frames

# Slider and controls
steps = [{
    "method": "animate",
    "label": str(yr),
    "args": [[str(yr)], {"mode": "immediate", "frame": {"duration": 0, "redraw": True}, "transition": {"duration": 0}}],
} for yr in years]

sliders = [{
    "active": 0,
    "currentvalue": {"prefix": "Year: ", "font": {"size": 14}},
    "pad": {"t": 10, "b": 10},
    "len": 0.9,
    "x": 0.05,
    "xanchor": "left",
    "y": -0.07,
    "yanchor": "top",
    "steps": steps
}]

updatemenus = [{
    "type": "buttons",
    "direction": "left",
    "x": 0.05, "y": -0.15,
    "pad": {"r": 10, "t": 10},
    "buttons": [
        {"label": "Play",  "method": "animate", "args": [None, {"fromcurrent": True, "frame": {"duration": 600, "redraw": True}, "transition": {"duration": 200}}]},
        {"label": "Pause", "method": "animate", "args": [[None], {"mode": "immediate", "frame": {"duration": 0, "redraw": False}, "transition": {"duration": 0}}]}
    ]
}]

# Layout
fig.update_layout(
    barmode="group",
    legend=dict(orientation="h", yanchor="bottom", y=-0.25, x=0.5, xanchor="center", title_text="Region"),
    sliders=sliders,
    updatemenus=updatemenus,
    margin=dict(l=50, r=20, t=70, b=120),
    height=720 if has_wages else 520,
    title_text="Income and Wage Distribution Comparison — Austin/Texas vs. United States",
    title_x=0.5,
)

# Axes
fig.update_yaxes(title_text="Share of Households", tickformat=".0%", row=1, col=1)
fig.update_xaxes(title_text="Income Bracket", row=1, col=1)

if has_wages:
    fig.update_yaxes(title_text="Share of Workers", tickformat=".0%", row=2, col=1)
    fig.update_xaxes(title_text="Wage Bracket", row=2, col=1)

# Annotations (insights)
fig.add_annotation(
    x=0.98, y=0.95, xref="paper", yref="paper", showarrow=False, align="right",
    text="Austin’s higher $200k+ share reflects wealth concentration", font=dict(color="#333", size=12), bgcolor="rgba(255,255,255,0.6)"
)
fig.add_annotation(
    x=0.02, y=0.38 if has_wages else 0.10, xref="paper", yref="paper", showarrow=False, align="left",
    text=("Texas wages are more concentrated in lower brackets" if has_wages else
          "Add wage CSV (Texas vs U.S.) to enable bottom panel"),
    font=dict(color="#333", size=12), bgcolor="rgba(255,255,255,0.6)"
)

fig

TypeError: make_subplots() got unexpected keyword argument(s): ['shared_x']